In [1]:
from google.colab import files

uploaded = files.upload()

uploaded.keys()

Saving stance_examples1.csv to stance_examples1.csv


dict_keys(['stance_examples1.csv'])

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, numpy as np, pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

df = pd.read_csv('stance_examples1.csv', encoding='latin1')
df['label'] = df['stance'].map({'con':0, 'pro':1})
df['text'] = df.apply(lambda r: f"[TOPIC] {r['topic']} [SEP] {r['sentence']}", axis=1)

# Make sure to adjust this path to where your 'stance-distilbert' model is located in your Google Drive.
model_path = '/content/drive/MyDrive/stance-distilbert'

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
id2label = {0:'con', 1:'pro'}

def classify_stance(sentence:str, topic:str):
    text = f"[TOPIC] {topic} [SEP] {sentence}"
    enc = tokenizer(text, truncation=True, padding=True, max_length=256, return_tensors='pt')
    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.softmax(logits, dim=-1).squeeze().tolist()
        pred = int(torch.argmax(logits, dim=-1))
    return {'stance': id2label[pred], 'proba_con': round(probs[0],3), 'proba_pro': round(probs[1],3)}

# enter the sentences
topic = input('enter the topic: ')
while True:
    sentence = input('enter the sources: ')
    if sentence == 'end':
        break
    print(classify_stance(sentence, topic))


enter the topic: Gene editing should be allowed
enter the sources: Gene editing, especially germline editing (changing genes passed to offspring), faces calls for bans due to significant safety concerns (off-target edits, cancer risks), profound ethical issues (altering human heredity, "designer babies," consent for future generations), potential for worsening social inequality (only the rich benefit), and fears of "playing God" or unintended evolutionary consequences, with many advocating strict regulation or moratoria until risks are understood. 
{'stance': 'con', 'proba_con': 0.538, 'proba_pro': 0.462}
enter the sources: end
